In [ ]:
#!/usr/bin/env python3
# Creates figures from mttr_events.csv
# Outputs: mttr_hist.png, mttr_ecdf.png, mttr_box_by_rule.png, mttr_bar_summary.png, mttr_summary.csv

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

CSV = Path("mttr_events.csv")
if not CSV.exists():
    raise SystemExit("mttr_events.csv not found. Run your MTTR script first.")

df = pd.read_csv(CSV, parse_dates=["detected_at", "responded_at"])
df["mttr_seconds"] = df["mttr_seconds"].astype(float)

df = df.sort_values("detected_at")
dup_mask = (df["rule"].shift(1).eq(df["rule"]) & (df["detected_at"] - df["detected_at"].shift(1)).dt.total_seconds().abs().lt(0.2))
df_nodup = df[~dup_mask].copy()

# ---- 1) Histogram ----
vals = df_nodup["mttr_seconds"].values
vals = vals[~np.isnan(vals)]
if len(vals) >= 1:
    bins = np.unique(np.logspace(np.log10(max(vals.min(), 0.1)), np.log10(max(vals.max(), 1.0)), 30))
    plt.figure(figsize=(7,4.5))
    plt.hist(vals, bins=bins)
    plt.xscale("log")
    plt.xlabel("MTTR (seconds, log scale)")
    plt.ylabel("Count of events")
    plt.title("MTTR Distribution (log scale)")
    plt.tight_layout()
    plt.savefig("mttr_hist.png", dpi=180)
    plt.close()

# ---- 2) ECDF (empirical CDF) ----
vals_sorted = np.sort(vals)
y = np.arange(1, len(vals_sorted)+1) / len(vals_sorted)
plt.figure(figsize=(7,4.5))
plt.plot(vals_sorted, y, drawstyle="steps-post")
plt.xlabel("MTTR (seconds)")
plt.ylabel("Cumulative fraction of events")
plt.title("MTTR ECDF")
plt.grid(True, which="both", axis="both", alpha=0.3)
plt.tight_layout()
plt.savefig("mttr_ecdf.png", dpi=180)
plt.close()

# ---- 3) Boxplot by rule (top N most frequent rules) ----
topN = 6
top_rules = df_nodup["rule"].value_counts().head(topN).index.tolist()
by_rule = [df_nodup.loc[df_nodup["rule"]==r, "mttr_seconds"].values for r in top_rules]
plt.figure(figsize=(10,5))
plt.boxplot(by_rule, labels=top_rules, showfliers=False)
plt.ylabel("MTTR (seconds)")
plt.title(f"MTTR by Rule (top {topN} rules)")
plt.xticks(rotation=15, ha="right")
plt.tight_layout()
plt.savefig("mttr_box_by_rule.png", dpi=180)
plt.close()

# ---- 4) Summary bars: mean/median/p90 across rules ----
def p90(x): return np.percentile(x, 90) if len(x) else np.nan
summary = (df_nodup
           .groupby("rule")["mttr_seconds"]
           .agg(["count","mean","median",p90,"max"])
           .sort_values("count", ascending=False))
summary.to_csv("mttr_summary.csv", float_format="%.3f")
topS = summary.head(topN)
x = np.arange(len(topS))
w = 0.3
plt.figure(figsize=(10,5))
plt.bar(x - w, topS["mean"].values, width=w)
plt.bar(x,      topS["median"].values, width=w)
plt.bar(x + w,  topS["p90"].values, width=w)
plt.xticks(x, topS.index, rotation=15, ha="right")
plt.ylabel("Seconds")
plt.title("MTTR per Rule (mean / median / p90)")
plt.legend(["mean","median","p90"])
plt.tight_layout()
plt.savefig("mttr_bar_summary.png", dpi=180)
plt.close()

print("Wrote: mttr_hist.png, mttr_ecdf.png, mttr_box_by_rule.png, mttr_bar_summary.png, mttr_summary.csv")


/tmp/ipython-input-869499226.py:56: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(by_rule, labels=top_rules, showfliers=False)


Wrote: mttr_hist.png, mttr_ecdf.png, mttr_box_by_rule.png, mttr_bar_summary.png, mttr_summary.csv
